In [13]:
# Importamos bibliotecas
import numpy as np
import pandas as pd
from datetime import datetime
import re
from sklearn.ensemble import IsolationForest

# Establecemos las columnas
colz=["IP","timestamp","status code","bytes sent","user agent"]

li=[]
c=0

# Abrimos el archivo
with open("logs/access.log") as f:
    while(True):
        if(c==100000):
            break
        if(c%10000==0):
            print(c) # Esto nose, yo lo borraria
        try:
            x=f.readline()
            if x==None:
                break
            log=x.split(" ")
            if(len(log)<12):
                continue
            #print([log[0],log[3].strip('['),log[5].strip('"'),log[7].strip('"'),int(log[8]),int(log[9]),' '.join(log[11:-1]).strip('"')])
            li.append([log[0],log[3].strip('['),int(log[8]),int(log[9]),' '.join(log[11:-1]).strip('"')])
        except Exception as e:
            pass
            #print(e)
            #print(log)
            #print(int(log[8]),int(log[9]))
            #print("skipped an entry")
        c+=1

# basic feature engineering
logs_df=pd.DataFrame(li,columns=colz)
logs_df["timestamp"]=pd.to_datetime(logs_df["timestamp"],format='%d/%b/%Y:%H:%M:%S')

#logs_df["url_length"]=logs_df["URL"].apply(len)
#logs_df["url_depth"]=logs_df["URL"].apply(lambda x : x.count('/'))
#logs_df['num_encoded_chars'] = logs_df['URL'].apply(lambda x: len(re.findall(r'%[0-9A-Fa-f]{2}', x)))
#logs_df['num_special_chars'] = logs_df['URL'].apply(lambda x: len(re.findall(r'[|,;]', x)))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [14]:
logs_df['IP'] = logs_df['IP'].astype(str).str.strip()

In [15]:
from tqdm import tqdm
import geoip2.database

reader = geoip2.database.Reader('GeoLite2-City.mmdb')

print("Geolocalizando IPs...")
continents, countries, cities, latitudes, longitudes = [], [], [], [], []

for ip in tqdm(logs_df['IP']):
    try:
        response = reader.city(ip)
        continents.append(response.continent.name or 'Privada/Local')
        countries.append(response.country.name or 'Privada/Local')
        cities.append(response.city.name or 'Privada/Local')
        latitudes.append(response.location.latitude if response.location.latitude else None)
        longitudes.append(response.location.longitude if response.location.longitude else None)
    except:
        continents.append('Privada/Local')
        countries.append('Privada/Local')
        cities.append('Privada/Local')
        latitudes.append(None)
        longitudes.append(None)

logs_df['continente'] = continents
logs_df['pais'] = countries
logs_df['ciudad'] = cities
logs_df['latitud'] = latitudes
logs_df['longitud'] = longitudes

reader.close()

print("¡Geolocalización completada!")
print(logs_df[['IP', 'continente', 'pais', 'ciudad', 'latitud', 'longitud']].head())


Geolocalizando IPs...


100%|██████████| 99997/99997 [00:05<00:00, 16669.96it/s]


¡Geolocalización completada!
              IP     continente             pais         ciudad  latitud  \
0   54.36.149.41         Europe           France  Privada/Local  48.8582   
1    31.56.96.51         Europe  The Netherlands      Amsterdam  52.3759   
2    31.56.96.51         Europe  The Netherlands      Amsterdam  52.3759   
3  40.77.167.129  North America    United States        Boydton  36.6694   
4    91.99.72.15         Europe          Germany      Nuremberg  49.4527   

   longitud  
0    2.3387  
1    4.8975  
2    4.8975  
3  -78.3877  
4   11.0783  


In [16]:
# Nos quedamos solamente con las variables que nos interesan
ubicaciones = logs_df[["continente" , "pais", "ciudad", "latitud", "longitud"]]

# Eliminamos duplicados
ubicaciones = ubicaciones.drop_duplicates().reset_index(drop=True)

# Añadimos el ID
#ubicaciones.insert(0, "id_ubicacion", range(1, len(ubicaciones) + 1))

In [17]:
ubicaciones

,continente,pais,ciudad,latitud,longitud
0,Europe,France,Privada/Local,48.8582,2.3387
1,Europe,The Netherlands,Amsterdam,52.3759,4.8975
2,North America,United States,Boydton,36.6694,-78.3877
3,Europe,Germany,Nuremberg,49.4527,11.0783
4,North America,United States,Privada/Local,37.7510,-97.8220
...,...,...,...,...,...
194,North America,United States,Mission Viejo,33.6128,-117.6622
195,Europe,Poland,Warsaw,52.2319,20.9943
196,Asia,Hong Kong,Wan Chai,22.2848,114.1703
197,Asia,Türkiye,Ankara,39.9207,32.8540
